In [1]:
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import os
import base64

In [2]:
# Gmail API Configuration
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']
CREDENTIALS_FILE = '../credentials/credentials.json'
TOKEN_FILE = 'token.json'

In [19]:
class GmailHandler:
    """Minimal Gmail fetcher"""

    def __init__(self):
        self.service = None
        self.authenticate()

    def authenticate(self):
        """Authenticate with Gmail API"""
        creds = None

        # Load existing token
        if os.path.exists(TOKEN_FILE):
            creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
            print("creds", creds.to_json())

        # If no valid token, log in
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
                creds = flow.run_local_server(port=0)
            # Save token
            with open(TOKEN_FILE, 'w') as token:
                token.write(creds.to_json())

        self.service = build('gmail', 'v1', credentials=creds)

    def get_labels(self):
        """Fetch and display only user-created Gmail labels"""
        try:
            results = self.service.users().labels().list(userId='me').execute()
            labels = results.get('labels', [])

            # Filter only user-created labels
            user_labels = [label for label in labels if label.get('type') == 'user']

            if not user_labels:
                print("⚠️ No user-created labels found.")
                return []

            print("🏷️  Your Gmail Labels (created by you):")
            for label in user_labels:
                print(f" - {label['name']}")
            print()
            return user_labels

        except Exception as e:
            print(f"❌ Error fetching labels: {e}")
            return []
        
    def get_unread_emails(self, max_results=50):
        """Fetch unread emails"""
        try:
            results = self.service.users().messages().list(
                userId='me',
                labelIds=['INBOX'],
                q='is:unread',
                maxResults=max_results
            ).execute()

            messages = results.get('messages', [])
            if not messages:
                print("✅ No unread emails found.")
                return []

            print(f"📬 Found {len(messages)} unread emails:\n")
            emails = []
            for msg in messages:
                details = self.get_email_details(msg['id'])
                if details:
                    emails.append(details)
            return emails

        except Exception as e:
            print(f"❌ Error fetching emails: {e}")
            return []
        
    def get_email_details(self, msg_id):
        """Get full email details + labels"""
        try:
            msg = self.service.users().messages().get(
                userId='me', id=msg_id, format='full'
            ).execute()

            headers = msg['payload']['headers']
            subject = next((h['value'] for h in headers if h['name'] == 'Subject'), '(No Subject)')
            sender = next((h['value'] for h in headers if h['name'] == 'From'), '(Unknown)')
            snippet = msg.get('snippet', '')
            labels = msg.get('labelIds', [])

            print(f"📧 From: {sender}")
            print(f"   Subject: {subject}")
            print(f"   Labels: {', '.join(labels) if labels else '(No Labels)'}")
            print(f"   Snippet: {snippet[:80]}...\n")

            return {"id": msg_id, "from": sender, "subject": subject, "labels": labels, "snippet": snippet}

        except Exception as e:
            print(f"❌ Error reading email: {e}")
            return None



In [20]:
handler = GmailHandler()

creds {"token": "ya29.a0ATi6K2vFDxHz8YiXLCEtTHIridxe7JwFJgQG5HjMkz3jo2xJRXXxhTKNkIg1IqTkrA-YbvnvGo84cQkUGek1wtP6D9ll9LxJZPQJ_BYUc55kUdUM1jloI6BEY3IHMOYTfuUyNNvsodfA1dCs0K-RrwDWedzXC-or4dF96gJorJebP8idjg5ku_coHfhv0WSM1IzkKLoaCgYKAcUSARMSFQHGX2MifIkxrG9XlldBHYOAy_Dvng0206", "refresh_token": "1//05TrQi9MqXxoTCgYIARAAGAUSNwF-L9IrtueZ2vud33vXcr9aQhXc0R1T7ekq0Lv2GFgN_cvw8k0yDL-GLPc46lwwoOOwCdir4lc", "token_uri": "https://oauth2.googleapis.com/token", "client_id": "643813721017-2err0bl1c4ueihcvsgbqm7cckvulsip6.apps.googleusercontent.com", "client_secret": "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", "scopes": ["https://www.googleapis.com/auth/gmail.modify"], "universe_domain": "googleapis.com", "account": "", "expiry": "2025-11-13T05:14:17Z"}


In [21]:
handler.get_labels()

🏷️  Your Gmail Labels (created by you):
 - Ghosted
 - Rejection
 - Interview
 - applied1
 - NoUpdates



[{'id': 'Label_2264874397049143065', 'name': 'Ghosted', 'type': 'user'},
 {'id': 'Label_3665898797705156409', 'name': 'Rejection', 'type': 'user'},
 {'id': 'Label_5036635321915363753', 'name': 'Interview', 'type': 'user'},
 {'id': 'Label_5596382569500237253',
  'name': 'applied1',
  'messageListVisibility': 'show',
  'labelListVisibility': 'labelShow',
  'type': 'user'},
 {'id': 'Label_8471805027767072590', 'name': 'NoUpdates', 'type': 'user'}]

In [22]:
handler.get_unread_emails()

📬 Found 50 unread emails:

📧 From: Alcumus <alcumus@pinpoint.email>
   Subject: Thanks for Your Application!
   Labels: UNREAD, IMPORTANT, CATEGORY_UPDATES, INBOX
   Snippet: Thanks for Your Application! Hi Abhishek, Thanks for applying for the AI Enginee...

📧 From: LinkedIn Job Alerts <jobalerts-noreply@linkedin.com>
   Subject: “software engineer”: Cresta - Software Engineer, Backend and more
   Labels: UNREAD, IMPORTANT, CATEGORY_UPDATES, INBOX
   Snippet: Cresta Software Engineer, Backend: Cresta is on a mission to turn every customer...

📧 From: TD@myworkday.com
   Subject: TD Careers: Thank you for your job application! / Carrières à la TD : Merci pour votre candidature! – Software Engineer I ( R_1449707 )
   Labels: UNREAD, IMPORTANT, CATEGORY_UPDATES, INBOX
   Snippet: Version française ci-dessous. Hi Abhishek, We&#39;ve received your application f...

📧 From: LinkedIn Job Alerts <jobalerts-noreply@linkedin.com>
   Subject: “data analyst”: Wiraa - Data Scientist and more
   La

[{'id': '19a7b5636e018a37',
  'from': 'Alcumus <alcumus@pinpoint.email>',
  'subject': 'Thanks for Your Application!',
  'labels': ['UNREAD', 'IMPORTANT', 'CATEGORY_UPDATES', 'INBOX'],
  'snippet': 'Thanks for Your Application! Hi Abhishek, Thanks for applying for the AI Engineer position at Alcumus. Your application will be reviewed shortly by a member of our recruitment team; we do our best to'},
 {'id': '19a7b4689686659f',
  'from': 'LinkedIn Job Alerts <jobalerts-noreply@linkedin.com>',
  'subject': '“software engineer”: Cresta - Software Engineer, Backend and more',
  'labels': ['UNREAD', 'IMPORTANT', 'CATEGORY_UPDATES', 'INBOX'],
  'snippet': 'Cresta Software Engineer, Backend: Cresta is on a mission to turn every customer… ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏'},
 {'id': '19a7b0a30d7573ca',
  'from': 'TD@myworkday.com',
  'subject': 'TD Careers: Thank you for your job application! / Carrières à la 